## 配置文件

In [1]:
import sys
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
from risk_models.config.read_config.read_func import Read_Oracle
from risk_models.config.write_config.write_func import Write_Oracle
import datetime
import pandas as pd

## 库存料号盈亏模型

In [2]:
class StockModelSt2:
    def __init__(self, org_code, open_time, close_time):
        self.org_code = org_code
        self.open_time = open_time
        self.close_time = close_time
    
    def cleanData(self):
        # 分别读取期初库存head表/detail表（iscurrent = 1）
        STOCK_OPENING_HEAD = Read_Oracle().read_oracle(sql= """ select * from OPENING_INVENTORY where iscurrent = 1""", database='dbods')
        STOCK_OPENING_DETAIL = Read_Oracle().read_oracle(sql= """ select * from OPENING_INVENTORY_DETAIL where iscurrent = 1""", database='dbods')
        # 根据输入企业信用代码与开始时间，筛选期初库存表
        STOCK_OPENING_DETAIL_PID = int(STOCK_OPENING_HEAD[(STOCK_OPENING_HEAD['CREDIT_CODE']==self.org_code) & (STOCK_OPENING_HEAD['OPT_DATE']==self.open_time) ]['ID'])
        STOCK_OPENING_DETAIL = STOCK_OPENING_DETAIL[STOCK_OPENING_DETAIL['PID']==STOCK_OPENING_DETAIL_PID]
        # 转换“数量”为数字
        STOCK_OPENING_DETAIL['DCL_QTY']=STOCK_OPENING_DETAIL['DCL_QTY'].map(float)
        # 汇总期初库存表
        STOCK_OPENING_DETAIL=STOCK_OPENING_DETAIL.groupby(['GDS_MTNO'])['DCL_QTY'].sum().reset_index()
        STOCK_OPENING_DETAIL=STOCK_OPENING_DETAIL.rename(columns={'GDS_MTNO':'COP_G_NO'})
        
        # 读取出入库表（限制出入库类型）
        # ---- 测试用 ----
        #STOCK_BILL = Read_Oracle().read_oracle(sql= """ select * from ods_zmxpq.ems_stock_bill where CAPXACTION != 'D' and business_type in ('2','3','4') """, database='dbods')
        # ---- 生产用 ----
        STOCK_BILL = Read_Oracle().read_oracle(sql= """ select * from ods_zmxpq.ems_stock_bill where CAPXACTION != 'D' and bill_type in ('1','2','3','4','5','6','A','B') """, database='dbods')
        # 根据输入企业信用代码与起始时间，筛选出入库表
        STOCK_BILL = STOCK_BILL[(STOCK_BILL['ORG_CODE']==self.org_code) & (STOCK_BILL['ACTRUAL_STOCK_DATE']>=self.open_time) & (STOCK_BILL['ACTRUAL_STOCK_DATE']<=self.close_time)]
        # 转换“数量”为数字
        STOCK_BILL['QTY_CO']=STOCK_BILL['QTY_CO'].map(float)
        # 转换“出入库类型”字段数据类型
        STOCK_BILL['STOCK_BILL_TYPE']=STOCK_BILL['STOCK_BILL_TYPE'].map(int)
        # 对"出入库类型"进行转化便于计算
        STOCK_BILL['STOCK_BILL_TYPE']=STOCK_BILL['STOCK_BILL_TYPE'].map(lambda x: -1 if x ==2 else 1)
        # 将出库的数量变为负数
        STOCK_BILL['QTY']=STOCK_BILL['QTY'] * STOCK_BILL['STOCK_BILL_TYPE']
        # 汇总出入库表
        STOCK_BILL=STOCK_BILL.groupby(['COP_G_NO'])['QTY_CO'].sum().reset_index()
        
        # 汇总期初和出入库表，生成期末库存表
        STOCK_END_DETAIL = pd.merge(STOCK_BILL, STOCK_OPENING_DETAIL, how='left', on = ['COP_G_NO'])
        STOCK_END_DETAIL = STOCK_END_DETAIL.rename(columns={'QTY_CO':'QTY_CHANGE','DCL_QTY':'QTY_BEFORE'})
        # 将期初库存为空的行值设为0
        STOCK_END_DETAIL = STOCK_END_DETAIL.fillna(0)
        # 计算期末库存
        STOCK_END_DETAIL['QTY_AFTER'] = STOCK_END_DETAIL['QTY_CHANGE'] + STOCK_END_DETAIL['QTY_BEFORE']
        
        
        # 加入企业信息; 期初期末时间；ID; 模型运行时间
        STOCK_END_DETAIL['ORG_CODE'] = self.org_code
        STOCK_END_DETAIL['STARTDT'] = self.open_time
        STOCK_END_DETAIL['ENDDT'] = self.close_time
        STOCK_END_DETAIL['ID'] = range(len(STOCK_END_DETAIL))
        detail_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        STOCK_END_DETAIL['CHECK_TIME'] = datetime.datetime.strptime(detail_now, "%Y-%m-%d %H:%M:%S")
        
        # 重新排序
        STOCK_END_DETAIL = STOCK_END_DETAIL[['ID','ORG_CODE','STARTDT','ENDDT','COP_G_NO','QTY_BEFORE','QTY_CHANGE','QTY_AFTER','CHECK_TIME']]
        
        # 判断结果是否为空：为空只返回table，不为空写入数据库
        if STOCK_END_DETAIL.empty :
            return STOCK_END_DETAIL
        else:
            Write_Oracle().write_oracle('BD_RISK_DETAIL_STOCK_ST2',STOCK_END_DETAIL,org_code = self.org_code)
            return STOCK_END_DETAIL
    
    def calData(self):
        # 读取明细表
        STOCK_END_RESULT = Read_Oracle().read_oracle(sql= """ select * from BD_RISK_DETAIL_STOCK_ST2 where iscurrent = 1 """, database = 'dbods')
        # 筛选出制定企业的数据
        STOCK_END_RESULT = STOCK_END_RESULT[STOCK_END_RESULT['ORG_CODE'] == self.org_code]
        # 删去不用的列
        STOCK_END_RESULT.drop(columns = ['ID','CHECK_TIME','ISCURRENT','LASTUPDATE'],inplace=True)
        
        # 读取参数表(待更新)
        # parm = Read_Oracle().read_oracle(sql= """ select * from config_table """, database='dbods')
        # 设置参数
        # if 参数为null:
            # cutoff = {'高阈值': STOCK_END_RESULT['QTY_AFTER'].quantile(q=0.8), '低阈值':float(0)}
            # rate = {'盈':float(-50 / len(STOCK_END_RESULT)), '亏':float(-100 / len(STOCK_END_RESULT))}
        # else:
            # 读取参数表参数
        
        # 根据期末库存值设定阈值
        cutoff = {'高阈值': STOCK_END_RESULT['QTY_AFTER'].quantile(q=0.8), '低阈值':float(0)}
        # 根据数据量设定惩罚系数（库存过高暂不扣分）
        ratio = {'盈':float(0), '亏':float(-100 / len(STOCK_END_RESULT))}
        
        # 通过阈值计算标签
        STOCK_END_RESULT['RISK_LABEL'] = STOCK_END_RESULT['QTY_AFTER'].map(lambda x: '库存过高' if x >= cutoff['高阈值'] and x >0 else ('负库存' if x < cutoff['低阈值'] else '正常'))
        # 计算分数
        STOCK_END_RESULT['SCORE'] = STOCK_END_RESULT['RISK_LABEL'].map(lambda x: ratio['盈'] if x == '库存过高' else (ratio['亏'] if x == '负库存' else 0))
        STOCK_END_SCORE = round(100 + STOCK_END_RESULT['SCORE'].sum(), 2)
        
        # 更新ID和运行时间
        STOCK_END_RESULT['ID'] = range(len(STOCK_END_RESULT))
        result_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        STOCK_END_RESULT['CHECK_TIME'] = datetime.datetime.strptime(result_now, "%Y-%m-%d %H:%M:%S")
        
        # 删去不用的列
        STOCK_END_RESULT.drop(columns = ['QTY_BEFORE','QTY_CHANGE','QTY_AFTER'],inplace=True)
        
        # 重新排序
        STOCK_END_RESULT = STOCK_END_RESULT[['ID','ORG_CODE','STARTDT','ENDDT','COP_G_NO','RISK_LABEL','SCORE','CHECK_TIME']]
        
        # 判断结果是否为空：为空只返回table，不为空写入数据库
        if STOCK_END_RESULT.empty :
            return STOCK_END_RESULT, STOCK_END_SCORE
        else:
            Write_Oracle().write_oracle('BD_RISK_RESULT_STOCK_ST2',STOCK_END_RESULT, org_code = self.org_code)
            return STOCK_END_RESULT, STOCK_END_SCORE
        

## 测试对象（参数：企业信用代码；期初期末时间）

In [3]:
STOCK_BIN_Model = StockModelSt2('91310000132612172J',datetime.datetime.strptime('2020-01-01', "%Y-%m-%d"),datetime.datetime.strptime('2021-12-31', "%Y-%m-%d"))

In [4]:
STOCK_END_DETAIL = STOCK_BIN_Model.cleanData()
STOCK_END_RESULT,STOCK_END_SCORE = STOCK_BIN_Model.calData()

2021-04-30 15:25:10.264 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.519s
2021-04-30 15:25:14.021 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 3.748s
2021-04-30 15:25:27.327 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 12.67s
2021-04-30 15:25:30.200 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 1.175s
2021-04-30 15:25:30.513 | INFO     | risk_models.config.write_config.write_func:write_oracle:128 - Processing... Writing 5224 rows into database
2021-04-30 15:25:30.997 | INFO     | risk_models.config.write_config.write_func:write_oracle:135 - Insert data into BD_RISK_DETAIL_STOCK_ST2 successfully! Total write time spent 1.979s
2021-04-30 15:25:31.222 | INFO     | risk_models.config.read_confi

## 明细表

In [5]:
STOCK_END_DETAIL

,ID,ORG_CODE,STARTDT,ENDDT,COP_G_NO,QTY_BEFORE,QTY_CHANGE,QTY_AFTER,CHECK_TIME
0,0,91310000132612172J,2020-01-01,2021-12-31,000000110000000495,0.0,11.000,11.000,2021-04-30 15:25:28
1,1,91310000132612172J,2020-01-01,2021-12-31,000000110000000499,0.0,4.004,4.004,2021-04-30 15:25:28
2,2,91310000132612172J,2020-01-01,2021-12-31,000000110000000500,0.0,13.900,13.900,2021-04-30 15:25:28
3,3,91310000132612172J,2020-01-01,2021-12-31,000000110000000503,0.0,36.000,36.000,2021-04-30 15:25:28
4,4,91310000132612172J,2020-01-01,2021-12-31,000000110000000505,0.0,4.500,4.500,2021-04-30 15:25:28
...,...,...,...,...,...,...,...,...,...
5219,5219,91310000132612172J,2020-01-01,2021-12-31,000000410000116009,0.0,2.000,2.000,2021-04-30 15:25:28
5220,5220,91310000132612172J,2020-01-01,2021-12-31,000000410000123014,0.0,2.000,2.000,2021-04-30 15:25:28
5221,5221,91310000132612172J,2020-01-01,2021-12-31,000000410000126819,0.0,2.000,2.000,2021-04-30 15:25:28
5222,5222,91310000132612172J,2020-01-01,2021-12-31,000000410000153549,0.0,2.000,2.000,2021-04-30 15:25:28


## 结果表

In [6]:
STOCK_END_RESULT

,ID,ORG_CODE,STARTDT,ENDDT,COP_G_NO,RISK_LABEL,SCORE,CHECK_TIME
0,0,91310000132612172J,2020-01-01,2021-12-31,000000110000000495,正常,0.0,2021-04-30 15:25:31
1,1,91310000132612172J,2020-01-01,2021-12-31,000000110000000499,正常,0.0,2021-04-30 15:25:31
2,2,91310000132612172J,2020-01-01,2021-12-31,000000110000000500,正常,0.0,2021-04-30 15:25:31
3,3,91310000132612172J,2020-01-01,2021-12-31,000000110000000503,正常,0.0,2021-04-30 15:25:31
4,4,91310000132612172J,2020-01-01,2021-12-31,000000110000000505,正常,0.0,2021-04-30 15:25:31
...,...,...,...,...,...,...,...,...
5219,5219,91310000132612172J,2020-01-01,2021-12-31,000000410000116009,正常,0.0,2021-04-30 15:25:31
5220,5220,91310000132612172J,2020-01-01,2021-12-31,000000410000123014,正常,0.0,2021-04-30 15:25:31
5221,5221,91310000132612172J,2020-01-01,2021-12-31,000000410000126819,正常,0.0,2021-04-30 15:25:31
5222,5222,91310000132612172J,2020-01-01,2021-12-31,000000410000153549,正常,0.0,2021-04-30 15:25:31


## 分数

In [7]:
STOCK_END_SCORE

100.0